# Univariate CNN Models

# Imports + Load Data

In [1]:
import math

import numpy as np
import pandas as pd

from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.layers.convolutional import Conv1D 
from sklearn.metrics import mean_squared_error
from keras.layers.convolutional import MaxPooling1D

2023-07-01 11:11:17.455414: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
PATH_TO_DATASETS = '../datasets/'
X_voo_dataset = PATH_TO_DATASETS + 'X-voo.csv'
y_voo_dataset = PATH_TO_DATASETS + 'y-voo.csv'

In [3]:
X = pd.read_csv(X_voo_dataset)
y = pd.read_csv(y_voo_dataset)

In [4]:
X = X.loc[0:, ['t-3', 't-2', 't-1']]

In [5]:
X

,t-3,t-2,t-1
0,437.929993,440.579987,438.779999
1,440.579987,438.779999,430.200012
2,438.779999,430.200012,430.209991
3,430.200012,430.209991,425.380005
4,430.209991,425.380005,427.679993
...,...,...,...
361,395.980011,400.019989,401.609985
362,400.019989,401.609985,401.000000
363,401.609985,401.000000,408.690002
364,401.000000,408.690002,403.489990


In [6]:
y = y.loc[0:, 't']

In [7]:
y

0      430.200012
1      430.209991
2      425.380005
3      427.679993
4      433.559998
          ...    
361    401.000000
362    408.690002
363    403.489990
364    402.309998
365    400.279999
Name: t, Length: 366, dtype: float64

In [8]:
prior_observations, forecasting_step = [3, 1]

# Create Keras model, fit, and predict

In [9]:
def univariate_cnn_model(prior_observations, forecasting_step, n_features=1):
    """
    """
    
    model = Sequential()

    conv1d_layer = Conv1D(filters=2, kernel_size=2, activation='relu', input_shape=(prior_observations, n_features), name="conv1d_layer")
    model.add(conv1d_layer)
    input_weights_hidden = conv1d_layer.get_weights()

    max_pooling_layer = MaxPooling1D(pool_size=2, strides=1, name="max_pooling_layer")
    model.add(max_pooling_layer)

    flatten_layer = Flatten(name="flatten_layer")
    model.add(flatten_layer)

    hidden_layer = Dense(5, activation='relu', name="hidden_layer")
    model.add(hidden_layer)

    output_layer = Dense(forecasting_step, name="output_layer")
    model.add(output_layer)

    model.compile(optimizer='adam', loss='mse')
    return model, model.summary()

In [10]:
ytrue = [100]

In [11]:
cnn_model, model_defined = univariate_cnn_model(prior_observations, forecasting_step)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_layer (Conv1D)       (None, 2, 2)              6         
                                                                 
 max_pooling_layer (MaxPooli  (None, 1, 2)             0         
 ng1D)                                                           
                                                                 
 flatten_layer (Flatten)     (None, 2)                 0         
                                                                 
 hidden_layer (Dense)        (None, 5)                 15        
                                                                 
 output_layer (Dense)        (None, 1)                 6         
                                                                 
Total params: 27
Trainable params: 27
Non-trainable params: 0
____________________________________________________________

2023-07-01 11:11:20.166892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
def fit_model(model_to_fit, X_train, y_train):
    model_to_fit.fit(X_train, y_train, epochs=2000, verbose=0)

    return model_to_fit

In [13]:
cnn_model_fitted = fit_model(cnn_model, X, y)

In [14]:
x_input = X.iloc[-1, -prior_observations:]
x_input

t-3    408.690002
t-2    403.489990
t-1    402.309998
Name: 365, dtype: float64

In [15]:
def predict(fitted_model, x_input, prior_observations):
        """
        Parameters:
        fitted_model -- keras model
        x_input -- pd series
        prior_observations -- int
        
        Returns
        -- int (of the prediction at the next time step(s))
        """
        x_input = x_input.values
        x_input = x_input.reshape((1, prior_observations))
        yhat = fitted_model.predict(x_input)
        
        return yhat

In [16]:
cnn_model_prediction = predict(cnn_model_fitted, x_input, prior_observations)

1/1 [==============================] - 0s 66ms/step


In [17]:
cnn_model_prediction

array([[23.937838]], dtype=float32)